In [112]:
import pandas as pd
import numpy as np

data = pd.read_csv("train.csv")

In [190]:
''' desbalanced classes '''

pd.value_counts(data["LotFrontage"].values)

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
65.0      44
85.0      40
78.0      25
21.0      23
90.0      23
68.0      19
24.0      19
64.0      19
73.0      18
72.0      17
79.0      17
63.0      17
55.0      17
100.0     16
51.0      15
66.0      15
74.0      15
52.0      14
59.0      13
71.0      12
82.0      12
40.0      12
43.0      12
67.0      12
57.0      12
        ... 
129.0      2
124.0      2
118.0      2
101.0      2
122.0      2
121.0      2
115.0      2
109.0      2
116.0      2
114.0      2
150.0      1
111.0      1
153.0      1
182.0      1
46.0       1
112.0      1
149.0      1
141.0      1
33.0       1
152.0      1
160.0      1
168.0      1
128.0      1
144.0      1
39.0       1
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Length: 110, dtype: int64

In [16]:
''' filter: minimun correlation with target '''

def corr_filter(data, target):
    cor = data.corr()
    cor_target = abs(cor[target])
    irrelevant_features = cor_target[cor_target < 0.6]
        
    data = data.drop(columns = irrelevant_features.index)
    
    return data

data = corr_filter(data, "SalePrice")

len(data.columns)


46

In [113]:
''' filter: NaN values '''

def nan_filter(data):
    
    nan_attr = {}
    sum_attr = data.isnull().sum()
        
    key = 0        
    for i in range(sum_attr.size):
        
        if(sum_attr.iloc[i] > 0):
                                    
            nan_attr[key] = [sum_attr.index[i], sum_attr.iloc[i]]
            key = key + 1
        
    for i in range(len(nan_attr)):
                
        if(nan_attr[i][1] >= 0.75 * data[nan_attr[i][0]].value_counts(dropna=False).sum()):
            data = data.drop(columns = nan_attr[i][0])
        elif(data[nan_attr[i][0]].dtype == 'object'):
            data[nan_attr[i][0]] = data[nan_attr[i][0]].fillna(data[nan_attr[i][0]].mode().iloc[0])
        else:
            data[nan_attr[i][0]] = data[nan_attr[i][0]].fillna(data[nan_attr[i][0]].mean())
    
    return data

data = nan_filter(data)
    

In [117]:
''' filter: desbalanced classes '''

def desc_filter(data):
    
    objects = []
    classes_attr = {}
    
    for i in range(len(data.columns)):
    
        if(data[data.iloc[0].index[i]].dtype == "object"):
            objects.append(data.iloc[0].index[i]) 
        
    for i in range(len(objects)):
        
        class_size = data[objects[i]].value_counts().size
        classes_attr[objects[i]] = []                    

        for j in range(class_size):
            
            classes_attr[objects[i]].append(data[objects[i]].value_counts()[j])   
            
    print(classes_attr)

desc_filter(data)


{'MSZoning': [1151, 218, 65, 16, 10], 'Street': [1454, 6], 'LotShape': [925, 484, 41, 10], 'LandContour': [1311, 63, 50, 36], 'Utilities': [1459, 1], 'LotConfig': [1052, 263, 94, 47, 4], 'LandSlope': [1382, 65, 13], 'Neighborhood': [225, 150, 113, 100, 86, 79, 77, 74, 73, 59, 58, 51, 49, 41, 38, 37, 28, 25, 25, 17, 17, 16, 11, 9, 2], 'Condition1': [1260, 81, 48, 26, 19, 11, 8, 5, 2], 'Condition2': [1445, 6, 2, 2, 2, 1, 1, 1], 'BldgType': [1220, 114, 52, 43, 31], 'HouseStyle': [726, 445, 154, 65, 37, 14, 11, 8], 'RoofStyle': [1141, 286, 13, 11, 7, 2], 'RoofMatl': [1434, 11, 6, 5, 1, 1, 1, 1], 'Exterior1st': [515, 222, 220, 206, 108, 61, 50, 26, 25, 20, 2, 2, 1, 1, 1], 'Exterior2nd': [504, 214, 207, 197, 142, 60, 38, 26, 25, 20, 10, 7, 5, 3, 1, 1], 'MasVnrType': [872, 445, 128, 15], 'ExterQual': [906, 488, 52, 14], 'ExterCond': [1282, 146, 28, 3, 1], 'Foundation': [647, 634, 146, 24, 6, 3], 'BsmtQual': [686, 618, 121, 35], 'BsmtCond': [1348, 65, 45, 2], 'BsmtExposure': [991, 221, 134, 11

In [48]:
''' changing value key '''

data['GarageQual'] = data['GarageQual'].map({'Ex': 5, 'Gd': 4, 'TA':3, 'Fa':2, 'Po': 1, "Na" : 0 })
pd.value_counts(data["GarageQual"].values)

3.0    1311
2.0      48
4.0      14
1.0       3
5.0       3
dtype: int64